In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!pip3 install git+https://github.com/huggingface/transformers

In [ ]:
!pip3 install googletrans

In [ ]:
!pip3 install openpyxl

In [ ]:
!pip3 install translate-api

In [ ]:
!pip3 install googletrans==4.0.0-rc1
#!pip3 install translators --upgrade --->doesn't work

In [ ]:
!pip3 install nltk

In [ ]:
!pip3 install -U pip setuptools wheel
!pip3 install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm
!python -m spacy download uk_core_news_sm

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
!pip install keybert

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data = pd.read_excel("https://github.com/winfreykong/BERT-Poem/blob/main/Data/Poems.xlsx?raw=true")
data.head()

,ID,Author of poem,Author of post,Themes curated,Theme(s) (comma separated),Theme 2 (blank),Theme 3 (blank),Language,In translation?,Original language (if post is a translation),Poem full text (copy and paste)
0,1,Halyna Kruk,Halyna Kruk,"{'maidan', 'revolution', 'protest', 'ukraine'}",Maidan,Ukraine,NaN,Ukrainian,No,NaN,"***\ncказати собі чесно, якомога чесніше,\nтак..."
1,2,Ostap Slyvynsky,Ostap Slyvynsky,"{'maidan', 'poetic form', 'haiku', 'revolution...","Haiku, Maidan",NaN,NaN,Ukrainian,No,NaN,ХАЙКУ З МАЙДАНУ\nДим у рукавах.\nНіби йдеш з ц...
2,3,Osip Mandelstam,Maria Stepanova,"{'time', 'war', 'twentieth century'}",war,twentieth century,NaN,Russian,No,NaN,(О.М.)\nНе мучнистой бабочкою белой\nВ землю я...
3,4,Boris Khersonsky,Boris Khersonsky,NaN,"Soviet Union, food, morphine, anthem",NaN,NaN,Russian,No,NaN,"***\n""Союз нерушимый"" сыграют и хором споют.\n..."
4,5,Oleksandr Irvanets,Oleksandr Irvanets,"{'seasons', 'time', 'new year'}",New year,NaN,NaN,Ukrainian,No,NaN,"***\nЦей рік, який почався з середи - \nКуди в..."


### Cleaning

In [ ]:
## Check percentage of null values
data.isnull().mean(axis=0)*100

ID                                               0.000000
Author of poem                                   0.000000
Author of post                                   0.000000
Themes curated                                  42.489627
Theme(s) (comma separated)                       6.887967
Theme 2 (blank)                                 88.713693
Theme 3 (blank)                                 94.107884
Language                                         0.331950
In translation?                                  0.082988
Original language (if post is a translation)    83.070539
Poem full text (copy and paste)                  0.000000
dtype: float64

In [ ]:
data = data.drop(columns=['Author of post', 'In translation?', 'Original language (if post is a translation)'])

In [ ]:
# Let's look at the types of languages we have in our dataset

# There are 4 null values in the Language column, we will drop them
data = data.dropna(subset=['Language'])
data['Language'].str.strip().str.split(", ").apply(set).apply(str).value_counts()

{'Ukrainian'}                                                                                                734
{'Russian'}                                                                                                  364
{'English'}                                                                                                   31
{'Polish'}                                                                                                    22
{'Hebrew'}                                                                                                     9
{'Ukrainian', 'Russian'}                                                                                       6
{'Belorusian'}                                                                                                 5
{'Italian'}                                                                                                    5
{'Estonian'}                                                                                    

In [ ]:
# Need to clean the following columns: themes, language and poem full text

#### Themes

# Notice that the column "Themes curated" consists of Map object. Here we convert themes-related columns to list type
data['Theme(s) (comma separated)']= data['Theme(s) (comma separated)'].str.split(",")
data['Themes curated']= data['Themes curated'].str.strip('}{').str.split(",")
# Now we combine all themes-related columns. We will skip 'Theme 2 (blank)' and 'Theme 3 (blank)' 
# since they mostly contain copies from other themes columns
data = data.assign(theme_combined=data['Theme(s) (comma separated)']+data['Themes curated'].fillna("").apply(list))

# Dropping all themes-related column, we will only use the "theme_combined" column moving forward
data.drop(columns=['Theme(s) (comma separated)', 'Themes curated', 'Theme 2 (blank)', 'Theme 3 (blank)'], inplace=True)

#### Language

# We only want to look at poems written in Russian or Ukrainian
russian_ukrainian = data.loc[data['Language'].str.split(", ").apply(set).apply(lambda x: ('Ukrainian' in x) or ('Russian' in x)), ]

# Notice that there are 6 poems written in both Russian and Ukrainian (skipping that for now - 15thFeb)

#### Poem text cleaning
def text_cleaning(text):
    text = text.replace("\n", ' ').replace('See less', '')
    text = re.sub('(#[\w\u0400-\u04FF]+)', '', text) #removing hashtags
    text = re.sub('[\[\(][\s\w\u0400-\u04FF\.,@$#!&\*\(\)\~\[\]]*[\]\)]', '', text) #removing brackets and everything inside
    return text

russian_ukrainian = russian_ukrainian.assign(light_clean_poem=russian_ukrainian['Poem full text (copy and paste)'].apply(text_cleaning))

In [ ]:
russian_ukrainian['Language'].value_counts()

Ukrainian                                                                                      734
Russian                                                                                        364
Russian, Ukrainian                                                                               6
Russian, Polish                                                                                  2
English, Ukrainian                                                                               1
Ukrainian, Belorusian                                                                            1
English, Russian                                                                                 1
English, Russian, Ukrainian, Polish, Belorusian, original Russian with various translations      1
Name: Language, dtype: int64

In [ ]:
russian_ukrainian.drop_duplicates(subset='Poem full text (copy and paste)')['Language'].value_counts()

Ukrainian                                                                                      718
Russian                                                                                        357
Russian, Ukrainian                                                                               6
Russian, Polish                                                                                  2
English, Ukrainian                                                                               1
Ukrainian, Belorusian                                                                            1
English, Russian                                                                                 1
English, Russian, Ukrainian, Polish, Belorusian, original Russian with various translations      1
Name: Language, dtype: int64

In [ ]:
## There are some duplicated poem entries. Let's take a look
check_dup = russian_ukrainian.duplicated(subset='Poem full text (copy and paste)', keep=False) # this returns a boolean series.
russian_ukrainian_dup = russian_ukrainian.loc[check_dup] # getting only duplicated rows
russian_ukrainian_dup

,ID,Author of poem,Language,Poem full text (copy and paste),theme_combined,light_clean_poem
39,40,Boris Khersonsky,Russian,"О.М.\nОн, кто армянское небо назвал близоруким...","[Mandelstam, poetry, gulag]","О.М. Он, кто армянское небо назвал близоруким,..."
40,41,Boris Khersonsky,Russian,"О.М.\nОн, кто армянское небо назвал близоруким...","[Mandelstam, Armenia, black sun, poetry]","О.М. Он, кто армянское небо назвал близоруким,..."
53,54,Boris Khersonsky,Russian,"***\nКто не любит свободу, те обычно - плечом ...","[freedom, struggle, protest]","*** Кто не любит свободу, те обычно - плечом к..."
54,55,Boris Khersonsky,Russian,"***\nКто не любит свободу, те обычно - плечом ...","[freedom, motherland, fear]","*** Кто не любит свободу, те обычно - плечом к..."
145,146,Halyna Kruk,Ukrainian,ранній Тичина - майже сто років тому:\nЗразу ж...,"[Maidan, 'maidan', 'revolution', 'protest']",ранній Тичина - майже сто років тому: Зразу ж ...
146,147,Pavlo Tychyna,Ukrainian,ранній Тичина - майже сто років тому:\nЗразу ж...,"[war, village, death, Sich, 'maidan', 'rev...",ранній Тичина - майже сто років тому: Зразу ж ...
173,174,Luba Iakymchuk,Ukrainian,це місто розлініяне в дощ\nяк тюремна роба\nна...,"[city, rain, 'maidan', 'seasons', 'revoluti...",це місто розлініяне в дощ як тюремна роба на т...
174,175,Luba Iakymchuk,Ukrainian,це місто розлініяне в дощ\nяк тюремна роба\nна...,"[Kyiv, Maidan, Rain, Music, 'maidan', 'sea...",це місто розлініяне в дощ як тюремна роба на т...
203,204,Serhiy Zhadan,Ukrainian,"* * * \nПливи, рибо, пливи – \nось твої остров...","[fish, life, love, death]","* * * Пливи, рибо, пливи – ось твої острови,..."
204,205,Serhii Zhadan,Ukrainian,"* * * \nПливи, рибо, пливи – \nось твої остров...","[fish, island, words, 'fish', 'bodies of wa...","* * * Пливи, рибо, пливи – ось твої острови,..."


Notice that some of the duplicated entries have different themes entries.

In [ ]:
grouped_dup = russian_ukrainian_dup.groupby('Poem full text (copy and paste)').agg({'theme_combined':'sum'}).reset_index()
russian_ukrainian.drop_duplicates(subset='Poem full text (copy and paste)', inplace=True)
russian_ukrainian = russian_ukrainian.merge(grouped_dup, left_on = 'Poem full text (copy and paste)', right_on = 'Poem full text (copy and paste)', how='left')

In [ ]:
russian_ukrainian['theme_combined_y'] = russian_ukrainian['theme_combined_y'].fillna(russian_ukrainian['theme_combined_x']) 
russian_ukrainian = russian_ukrainian.drop(columns='theme_combined_x')
russian_ukrainian.rename({'Author of poem':'poem_author', 'Language':'language', 'Poem full text (copy and paste)':'original_poem', 'theme_combined_y':'theme_combined'})

,ID,Author of poem,Language,Poem full text (copy and paste),light_clean_poem,theme_combined_y
0,1,Halyna Kruk,Ukrainian,"***\ncказати собі чесно, якомога чесніше,\nтак...","*** cказати собі чесно, якомога чесніше, так н...","[Maidan, 'maidan', 'revolution', 'protest', ..."
1,2,Ostap Slyvynsky,Ukrainian,ХАЙКУ З МАЙДАНУ\nДим у рукавах.\nНіби йдеш з ц...,ХАЙКУ З МАЙДАНУ Дим у рукавах. Ніби йдеш з цим...,"[Haiku, Maidan, 'maidan', 'poetic form', 'h..."
2,3,Osip Mandelstam,Russian,(О.М.)\nНе мучнистой бабочкою белой\nВ землю я...,Не мучнистой бабочкою белой В землю я заемный...,"[war, 'time', 'war', 'twentieth century']"
3,4,Boris Khersonsky,Russian,"***\n""Союз нерушимый"" сыграют и хором споют.\n...","*** ""Союз нерушимый"" сыграют и хором споют. В ...","[Soviet Union, food, morphine, anthem]"
4,5,Oleksandr Irvanets,Ukrainian,"***\nЦей рік, який почався з середи - \nКуди в...","*** Цей рік, який почався з середи - Куди він...","[New year, 'seasons', 'time', 'new year']"
...,...,...,...,...,...,...
1082,1201,Oleh Kotsarev,Ukrainian,СВІТ САМШИТ\n\nСвіте!\nХиткий самшите!\nГладжу...,СВІТ САМШИТ Світе! Хиткий самшите! Гладжу теб...,"[nature, art]"
1083,1202,Olena Huseinova,Ukrainian,Дорога Halyna Kruk :\nале з кожною розповіддю ...,Дорога Halyna Kruk : але з кожною розповіддю м...,"[Halyna Kruk, poetry, religion, Ukraine, w..."
1084,1203,Pavlo Korobchuk,Ukrainian,"- у мене закінчуються сили, я не в ресурсі\n- ...","- у мене закінчуються сили, я не в ресурсі - а...","[war, Ukraine, language, ammunition, repor..."
1085,1204,Victoria Amelina,Ukrainian,*\nдивне пекуче літо\nповне людей із моря\nїхн...,* дивне пекуче літо повне людей із моря їхні с...,"[Mariupol, war, summer, archive, memory, ..."


In [ ]:
russian_ukrainian = russian_ukrainian.rename(columns={'Author of poem':'poem_author', 'Language':'language', 'Poem full text (copy and paste)':'original_poem', 'theme_combined_y':'theme'})

In [ ]:
# Notice that there are a lot of duplicated themes in the theme list, e.g. Maidan, 'maidan'
# Here we define a function to clean the lists in the "theme" column
def clean_list(l):
    try:
        for i in range(len(l)):
            l[i] = l[i].lower().replace("'",'').strip()
        return set(l)
    except TypeError:
        return 

In [ ]:
russian_ukrainian

,ID,poem_author,language,original_poem,light_clean_poem,theme
0,1,Halyna Kruk,Ukrainian,"***\ncказати собі чесно, якомога чесніше,\nтак...","*** cказати собі чесно, якомога чесніше, так н...","[Maidan, 'maidan', 'revolution', 'protest', ..."
1,2,Ostap Slyvynsky,Ukrainian,ХАЙКУ З МАЙДАНУ\nДим у рукавах.\nНіби йдеш з ц...,ХАЙКУ З МАЙДАНУ Дим у рукавах. Ніби йдеш з цим...,"[Haiku, Maidan, 'maidan', 'poetic form', 'h..."
2,3,Osip Mandelstam,Russian,(О.М.)\nНе мучнистой бабочкою белой\nВ землю я...,Не мучнистой бабочкою белой В землю я заемный...,"[war, 'time', 'war', 'twentieth century']"
3,4,Boris Khersonsky,Russian,"***\n""Союз нерушимый"" сыграют и хором споют.\n...","*** ""Союз нерушимый"" сыграют и хором споют. В ...","[Soviet Union, food, morphine, anthem]"
4,5,Oleksandr Irvanets,Ukrainian,"***\nЦей рік, який почався з середи - \nКуди в...","*** Цей рік, який почався з середи - Куди він...","[New year, 'seasons', 'time', 'new year']"
...,...,...,...,...,...,...
1082,1201,Oleh Kotsarev,Ukrainian,СВІТ САМШИТ\n\nСвіте!\nХиткий самшите!\nГладжу...,СВІТ САМШИТ Світе! Хиткий самшите! Гладжу теб...,"[nature, art]"
1083,1202,Olena Huseinova,Ukrainian,Дорога Halyna Kruk :\nале з кожною розповіддю ...,Дорога Halyna Kruk : але з кожною розповіддю м...,"[Halyna Kruk, poetry, religion, Ukraine, w..."
1084,1203,Pavlo Korobchuk,Ukrainian,"- у мене закінчуються сили, я не в ресурсі\n- ...","- у мене закінчуються сили, я не в ресурсі - а...","[war, Ukraine, language, ammunition, repor..."
1085,1204,Victoria Amelina,Ukrainian,*\nдивне пекуче літо\nповне людей із моря\nїхн...,* дивне пекуче літо повне людей із моря їхні с...,"[Mariupol, war, summer, archive, memory, ..."


In [ ]:
print("Number of null entry in theme column: ", russian_ukrainian['theme'].apply(clean_list).isna().sum())
russian_ukrainian['theme'] = russian_ukrainian['theme'].apply(clean_list)
russian_ukrainian['theme']

Number of null entry in theme column:  77


0                  {ukraine, maidan, revolution, protest}
1       {revolution, poetic form, maidan, haiku, protest}
2                          {time, twentieth century, war}
3                  {food, soviet union, morphine, anthem}
4                               {time, new year, seasons}
                              ...                        
1082                                        {nature, art}
1083    {poetry, lazarus, religion, ukraine, halyna kr...
1084      {ukraine, reportage, war, ammunition, language}
1085    {mariupol, souvenirs, archive, summer, memory,...
1086                      {needle, speech, silence, pain}
Name: theme, Length: 1087, dtype: object

In [ ]:
russian_ukrainian.dropna(subset=['theme'], inplace=True)

In [ ]:
russian_ukrainian.head()

,ID,poem_author,language,original_poem,light_clean_poem,theme
0,1,Halyna Kruk,Ukrainian,"***\ncказати собі чесно, якомога чесніше,\nтак...","*** cказати собі чесно, якомога чесніше, так н...","{ukraine, maidan, revolution, protest}"
1,2,Ostap Slyvynsky,Ukrainian,ХАЙКУ З МАЙДАНУ\nДим у рукавах.\nНіби йдеш з ц...,ХАЙКУ З МАЙДАНУ Дим у рукавах. Ніби йдеш з цим...,"{revolution, poetic form, maidan, haiku, protest}"
2,3,Osip Mandelstam,Russian,(О.М.)\nНе мучнистой бабочкою белой\nВ землю я...,Не мучнистой бабочкою белой В землю я заемный...,"{time, twentieth century, war}"
3,4,Boris Khersonsky,Russian,"***\n""Союз нерушимый"" сыграют и хором споют.\n...","*** ""Союз нерушимый"" сыграют и хором споют. В ...","{food, soviet union, morphine, anthem}"
4,5,Oleksandr Irvanets,Ukrainian,"***\nЦей рік, який почався з середи - \nКуди в...","*** Цей рік, який почався з середи - Куди він...","{time, new year, seasons}"


### Let's look at the top 10 most frequently occured themes and the least frequently occured themes

In [ ]:
russian_ukrainian['theme'].explode().value_counts().head(10)

war          322
religion     125
ukraine      124
death        103
time          96
russia        85
geography     84
seasons       72
family        62
language      55
Name: theme, dtype: int64

In [ ]:
russian_ukrainian['theme'].explode().value_counts()[-10:]

isolation                1
four walls               1
maple                    1
comic strip form         1
singing in dark times    1
information              1
wwi                      1
the earth                1
early childhood          1
needle                   1
Name: theme, dtype: int64

#### We can see that the lesser frequently occured themes are usually very specific, e.g. winter-summer vs seasons, bad habits vs habits

In [ ]:
unique_theme = russian_ukrainian['theme'].explode().value_counts().index.tolist()

In [ ]:
unique_theme[:20] # We will generate a list of high-level themes based on the top 20 themes

['war',
 'religion',
 'ukraine',
 'death',
 'time',
 'russia',
 'geography',
 'seasons',
 'family',
 'language',
 'memory',
 'animals',
 'nature',
 'maidan',
 'revolution',
 'christianity',
 'protest',
 'poetry',
 'love',
 'poetic form']

## Generate high-level themes

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
big_theme = []
lemmatizer = WordNetLemmatizer()
for i in unique_theme[:50]:
    big_theme.append(lemmatizer.lemmatize(i))

In [ ]:
big_theme[:20]

['war',
 'religion',
 'ukraine',
 'death',
 'time',
 'russia',
 'geography',
 'season',
 'family',
 'language',
 'memory',
 'animal',
 'nature',
 'maidan',
 'revolution',
 'christianity',
 'protest',
 'poetry',
 'love',
 'poetic form']

In [ ]:
## Generating our own big theme (as discussed with Professor Glaser)
big_theme = ['war', 'religion', 'nation', 'death', 'time',
             'protest', 'geography', 'season', 'family', 'language',
             'maidan','memory', 'animal', 'nature', 'love', 'poetry',   
             'psychology', 'politics', 'life', 'fruit', 'gender', 'mythology'] 

theme_id = {}
for i, t in enumerate(big_theme):
    theme_id[t] = i
theme_id

{'war': 0,
 'religion': 1,
 'nation': 2,
 'death': 3,
 'time': 4,
 'protest': 5,
 'geography': 6,
 'season': 7,
 'family': 8,
 'language': 9,
 'maidan': 10,
 'memory': 11,
 'animal': 12,
 'nature': 13,
 'love': 14,
 'poetry': 15,
 'psychology': 16,
 'politics': 17,
 'life': 18,
 'fruit': 19,
 'gender': 20,
 'mythology': 21}

In [ ]:
' ' in 'the earth'

True

In [ ]:
'the-earth'.split('-')

['the', 'earth']

In [ ]:
lemmatizer = WordNetLemmatizer()

def get_big_theme(themes, big_themes):
    lemmatizer = WordNetLemmatizer()
    out = []
    for i in themes:
        if ' ' in i:
            i = i.split()
            for j in i:
                if lemmatizer.lemmatize(j) in big_themes:
                    out.append(lemmatizer.lemmatize(j))
        else:
            if lemmatizer.lemmatize(i) in big_themes:
                out.append(lemmatizer.lemmatize(i))
    if len(out) == 0:
        return themes
    return out

In [ ]:
big_theme_ser = russian_ukrainian['theme'].apply(lambda x: get_big_theme(x, theme_id))
big_theme_ser

0                            [maidan, protest]
1                            [maidan, protest]
2                                  [time, war]
3       {food, soviet union, morphine, anthem}
4                               [time, season]
                         ...                  
1082                                  [nature]
1083                   [poetry, religion, war]
1084                           [war, language]
1085                             [memory, war]
1086           {needle, speech, silence, pain}
Name: theme, Length: 1010, dtype: object

In [ ]:
np.mean(big_theme_ser.apply(lambda x: isinstance(x, set)))*100

21.386138613861387

#### Around 21% of the poems do not have a big theme associated to it - possibly due to different naming conventions, i.e. winter-summer to season . We will use similarity scoring to get the most similar big theme

In [ ]:
import spacy

In [ ]:
def find_most_similar_theme(themes, big_themes, model="en_core_web_sm"):
    nlp = spacy.load(model)
    threshold = 0.7
    out = []
    for i in themes:
        max_sim = 0
        max_theme = ''
        #if ('-' in i): i = i.split('-')
        #else: i = i.split()
        #for j in i:
        w1 = nlp(i)
        for k in big_themes:
            if w1.similarity(nlp(k)) > max_sim:
                max_sim = w1.similarity(nlp(k))
                max_theme = k
                term = w1
        if max_sim > threshold:
            out.append((term, max_sim, max_theme))
    return out

In [ ]:
# test
find_most_similar_theme({'morphine', 'food', 'soviet union', 'anthem'},theme_id)

# Would expect the following big theme(s) to be returned: nation, food

<ipython-input-43-3ec4511c6e4c>:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  if w1.similarity(nlp(k)) > max_sim:
<ipython-input-43-3ec4511c6e4c>:14: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger m

[(food, 0.8613927682622928, 'death'), (morphine, 0.7150524988521699, 'love')]

#### Using the small model, en_core_web_sm, does not give accurate scores. This makes sense since the similarity score is computed "based on the tagger, parser and NER" as stated in the warning message above. We will try using larger models

In [ ]:
# test using larger model
find_most_similar_theme({'morphine', 'food', 'soviet union', 'anthem'},theme_id, model="en_core_web_lg")

[]

#### Hmm.. the large model might be too strict for our use

### More about SpaCy Similarity

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
lemmatizer.lemmatize('poems')

'poem'

In [ ]:
w1 = nlp('pain')
w2 = nlp('emotion')
w1.similarity(w2)

0.5123366545298419

In [ ]:
w1 = nlp('christianity')
w2 = nlp('religion')
w1.similarity(w2)

0.8092649878089295

In [ ]:
w1 = nlp('childhood')
w2 = nlp('memory')
w1.similarity(w2)

0.4691299983703067

In [ ]:
w1 = nlp('summer')
w2 = nlp('season')
w1.similarity(w2)

0.581541474951942

In [ ]:
w1 = nlp('ukraine')
w2 = nlp('country')
w1.similarity(w2)

0.3430874479439098

In [ ]:
w1 = nlp('christmas')
w2 = nlp('time')
w1.similarity(w2)

0.20934955324979118

### WordNet Similarity

In [ ]:
from nltk.corpus import wordnet

In [ ]:
test = wordnet.synsets("soviet union")

In [ ]:
for i in wordnet.synsets('nation', pos='n'):
    for j in test:
        print(i, j, i.wup_similarity(j))

### Best course of action is to manually annotate the dataset to ensure the correctness in further usage
#### We will only re-annotate the 24% where none of the entries in the exisiting "themes" column is in the "big_theme" we have identified

In [ ]:
yes_big_theme = russian_ukrainian[big_theme_ser.apply(lambda x: not(isinstance(x, set)))]
no_big_theme = russian_ukrainian[big_theme_ser.apply(lambda x: isinstance(x, set))]
no_big_theme

,ID,poem_author,language,original_poem,light_clean_poem,theme
3,4,Boris Khersonsky,Russian,"***\n""Союз нерушимый"" сыграют и хором споют.\n...","*** ""Союз нерушимый"" сыграют и хором споют. В ...","{food, soviet union, morphine, anthem}"
7,8,Oleh Kotsarev,Ukrainian,ПОВЕРНЕННЯ\n\nМісто L – так близько до міста N...,"ПОВЕРНЕННЯ Місто L – так близько до міста N, ...","{, city, return}"
13,14,Tadeusz Chabrowski,Ukrainian,Тадеуш Хабровський\nВуді Аллен каже\nбо в злі ...,Тадеуш Хабровський Вуді Аллен каже бо в злі ча...,"{theater, usa, woody allen}"
15,16,Oleksandr Irvanets,Ukrainian,А СВЯТО ТРИВАЄ...\n***\nНоворічна пісня рине \...,А СВЯТО ТРИВАЄ... *** Новорічна пісня рине Ві...,"{sasha grey, holiday, celebration, theater}"
18,19,Boris Khersonsky,Russian,"***\n\nПробудитесь, цари! Поднимайтесь скорей!...","*** Пробудитесь, цари! Поднимайтесь скорей! З...","{soviet union, christmas, revolution, imperial..."
...,...,...,...,...,...,...
1055,1169,Boris Khersonsky,Russian,"***\nИх рисовали втроем, спящих в одной постел...","*** Их рисовали втроем, спящих в одной постели...","{ekphrastic, christmas, star, angel}"
1056,1172,Halyna Kruk,Ukrainian,***\nщо там сьогодні в театрі тіней - \nвухаті...,*** що там сьогодні в театрі тіней - вухаті з...,"{light, shadow, theater, tiredness}"
1060,1176,Halyna Kruk,Ukrainian,невловимий Сковорода \nобходИв невмолимий світ...,невловимий Сковорода обходИв невмолимий світ ...,"{food, mundane, kitchen}"
1079,1197,Ivanna Skyba-Yakubova,Ukrainian,"***\nВисповідай мене, Апостоле.\nТут, на темно...","*** Висповідай мене, Апостоле. Тут, на темному...","{confession, apostle, christianity}"


In [ ]:
yes_big_theme = yes_big_theme.assign(big_theme=big_theme_ser[big_theme_ser.apply(lambda x: not(isinstance(x, set)))])
yes_big_theme

,ID,poem_author,language,original_poem,light_clean_poem,theme,big_theme
0,1,Halyna Kruk,Ukrainian,"***\ncказати собі чесно, якомога чесніше,\nтак...","*** cказати собі чесно, якомога чесніше, так н...","{ukraine, maidan, revolution, protest}","[maidan, protest]"
1,2,Ostap Slyvynsky,Ukrainian,ХАЙКУ З МАЙДАНУ\nДим у рукавах.\nНіби йдеш з ц...,ХАЙКУ З МАЙДАНУ Дим у рукавах. Ніби йдеш з цим...,"{revolution, poetic form, maidan, haiku, protest}","[maidan, protest]"
2,3,Osip Mandelstam,Russian,(О.М.)\nНе мучнистой бабочкою белой\nВ землю я...,Не мучнистой бабочкою белой В землю я заемный...,"{time, twentieth century, war}","[time, war]"
4,5,Oleksandr Irvanets,Ukrainian,"***\nЦей рік, який почався з середи - \nКуди в...","*** Цей рік, який почався з середи - Куди він...","{time, new year, seasons}","[time, season]"
5,6,Oleksandr Irvanets,Ukrainian,"ВІРШИК НА 1 СІЧНЯ \n(Обережно, жорстка лексика...","ВІРШИК НА 1 СІЧНЯ *** Здрастуй, Дєдушка Боду...","{grandpa bodun, family}",[family]
...,...,...,...,...,...,...,...
1081,1199,Kateryna Devdera,Ukrainian,"Хай це стане молитвою, хай стане твоєю силою,\...","Хай це стане молитвою, хай стане твоєю силою, ...","{jesus, death, god, christmas, prayer}",[death]
1082,1201,Oleh Kotsarev,Ukrainian,СВІТ САМШИТ\n\nСвіте!\nХиткий самшите!\nГладжу...,СВІТ САМШИТ Світе! Хиткий самшите! Гладжу теб...,"{nature, art}",[nature]
1083,1202,Olena Huseinova,Ukrainian,Дорога Halyna Kruk :\nале з кожною розповіддю ...,Дорога Halyna Kruk : але з кожною розповіддю м...,"{poetry, lazarus, religion, ukraine, halyna kr...","[poetry, religion, war]"
1084,1203,Pavlo Korobchuk,Ukrainian,"- у мене закінчуються сили, я не в ресурсі\n- ...","- у мене закінчуються сили, я не в ресурсі - а...","{ukraine, reportage, war, ammunition, language}","[war, language]"


In [ ]:
yes_big_theme.to_csv("yes_big_theme.csv")

In [ ]:
no_big_theme.to_csv('no_big_theme.csv')

# Translate

In [ ]:
from googletrans import Translator

In [ ]:
def translate(text):
    translator = Translator()
    return translator.translate(text, dest='en').text

# KeyBERT

In [ ]:
from keybert import KeyBERT

In [ ]:
kb_model = KeyBERT()

### Default KeyBERT uses "all-MiniLM-L6-v2"

In [ ]:
kb_model.extract_keywords(russian_ukrainian['light_clean_poem'].iloc[100])

[('сміятися', 0.3663),
 ('ніби', 0.3509),
 ('робила', 0.3446),
 ('порцеляни', 0.3403),
 ('вбивця', 0.3375)]

In [ ]:
russian_ukrainian.iloc[100]

ID                                                                108
poem_author                                             Oleh Kotsarev
language                                                    Ukrainian
original_poem       ТАЇНА\n\nВинятково смачна таємниця –\nчас, у я...
light_clean_poem    ТАЇНА  Винятково смачна таємниця – час, у яком...
theme                                                 {secrets, time}
Name: 105, dtype: object

In [ ]:
translate('ніби'), translate('робила'), translate('порцеляни'), translate('вбивця'), translate('сміятися')

('as if', 'did it', 'porcelain', 'The killer', 'laugh')

### Use multilingual BERT

In [ ]:
from transformers.pipelines import pipeline

multilingual_model = pipeline("feature-extraction", model="bert-base-multilingual-uncased")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
kb_model2 = KeyBERT(model=multilingual_model)

In [ ]:
key2 = kb_model2.extract_keywords(russian_ukrainian['light_clean_poem'].iloc[100])
key2

[('таємниць', 0.4042),
 ('різдвяна', 0.3899),
 ('починаємо', 0.371),
 ('перевернуте', 0.313),
 ('оголошення', 0.2948)]

In [ ]:
kw = [i[0] for i in key2]
kw

['таємниць', 'різдвяна', 'починаємо', 'перевернуте', 'оголошення']

In [ ]:
for i in kw:
    print(translate(i))

secrets
Christmas
We start
Turn it over
advertisement


In [ ]:
text = russian_ukrainian['light_clean_poem'].iloc[100]
text

'ТАЇНА  Винятково смачна таємниця – час, у якому всі переконані, що таємниць більше нема.  Я зараз дістану одне таке суспільство з шухлядки, яка ледь тримається знизу старого стола –  Обережно. Не розбити б. Ну, починаємо!  Пальцем об долоню тієї ж руки вибиваю те, що здається мені фламенко, люди в шухляді так гарно танцюють, що хочеться з’їсти їх розпалений мозок!  А зараз будуть мої улюблені персонажі цієї шухляди: перевернуте обличчя з двома яйцями ротів із порцеляни та одним зубатим оком, різдвяна ялинка-вбивця, що засліплює кордиор нічної лікарні, чоловік із діркою в шиї, крізь яку з шипінням виходить повітря, скліщені мурахи й ті, хто тремтить уві сні!  Всім іншим героям лишається тільки сміятися, вигадуючи їм дотепні прізвиська.  Я просто катаюсь підлогою з радості – й, мабуть, ось зараз почну їсти мізки в момент оголошення чергового прогнозу.  Та щось мене зупиняє, ховаю шухляду назад, танці тривають у темряві, як і решта ритуалів, замаскованих під торгівлю, ніби магма під обол

In [ ]:
import time

In [ ]:
def extract_keyword(model, text):
    return model.extract_keywords(text)

In [ ]:
start = time.time()
kw = extract_keyword(kb_model2, text)
end = time.time()
end-start

10.691414833068848

#### Takes around 10 seconds to extract keywords from a roughly 182 words poem

In [ ]:
kw_series = russian_ukrainian.head(10)['light_clean_poem'].apply(lambda x: extract_keyword(kb_model2, x))

In [ ]:
kw_series

0    [(покладаючись, 0.3382), (патріотичних, 0.2455...
1    [(пообіцяти, 0.4741), (пустимо, 0.3054), (вист...
2    [(восклицанья, 0.3814), (мыслящее, 0.2992), (м...
3    [(проснусь, 0.3366), (нерушимый, 0.331), (скры...
4    [(наступний, 0.4395), (четверга, 0.2653), (коч...
5    [(сніданок, 0.4183), (дєдушка, 0.4019), (вірши...
6    [(вдихаєш, 0.4907), (найглухішої, 0.4699), (на...
7    [(насправді, 0.3842), (ренесансовий, 0.3441), ...
8    [(європоньці, 0.3633), (краще, 0.262), (сидіти...
9    [(встигнемо, 0.419), (тиждень, 0.3152), (втоми...
Name: light_clean_poem, dtype: object

#### Around 1-2 minutes to extract keywords of the first 10 rows. Now let's try extracting keywords for the rest of our poems

In [ ]:
start = time.time()
kw_series = russian_ukrainian['light_clean_poem'].apply(lambda x: extract_keyword(kb_model2, x))
end = time.time()
end-start

6706.986086368561

In [ ]:
russian_ukrainian = russian_ukrainian.assign(keywords=kw_series)

In [ ]:
russian_ukrainian.to_csv('keyword_poems.csv')

#### 8730.58 seconds (around 2hr 25mins) to extract keywords of the entire dataset (without GPU)

#### 6707 seconds (around 1hr 50mins) to extract keywords of the entire dataset (with GPU!)

# Archived

In [ ]:
# get only Ukrainian
ukrainian = russian_ukrainian.loc[russian_ukrainian['Language'].str.split(", ").apply(lambda x: (len(x) == 1) and ('Ukrainian' in x) and ('Russian' not in x)), ]
ukrainian.shape[0]

718

In [ ]:
# get only Russian
russian = russian_ukrainian.loc[russian_ukrainian['Language'].str.split(", ").apply(lambda x: (len(x) == 1) and ('Ukrainian' not in x) and ('Russian' in x)), ]
russian.shape[0]

357